In [3]:
import dash
from dash import dcc, html, Input, Output, State, dash_table
import pandas as pd
import base64
import io

In [4]:
app = dash.Dash(__name__)

# App layout
app.layout = html.Div(children=[
    html.H1("F1 Races - Data Table Viewer", style={'textAlign': 'center'}),
    
    # Upload component
    html.Div(children=[
        dcc.Upload(
            id='upload-data',
            children=html.Div(children=['Drag and Drop or ', html.A('Select a File')]),
            style={
                'width': '100%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'textAlign': 'center',
                'margin': '10px'
            },
            multiple=False
        )
    ]),
    
    # Data table to display uploaded data
    html.Div(children=[
        dash_table.DataTable(
            id='data-table',
            style_table={'height': '400px', 'overflowY': 'auto'},
            style_cell={
                'textAlign': 'center',
                'whiteSpace': 'normal',
                'height': 'auto'
            },
            style_header={
                'fontWeight': 'bold',
                'backgroundColor': 'lightgrey'
            }
        )
    ])
])

# Callback to handle file upload and update the data table
@app.callback(
    Output('data-table', 'data'),
    Output('data-table', 'columns'),
    Input('upload-data', 'contents'),
    State('upload-data', 'filename')
)
def update_table(contents, filename):
    if contents is None:
        return [], []
    
    # Decode the uploaded file
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    
    try:
        # Read the uploaded CSV file
        df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
        
        # Return first few rows and column information for the table
        return df.head().to_dict('records'), [{"name": col, "id": col} for col in df.columns]
    
    except Exception as e:
        return [], []

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)